In [ ]:
# Copyright 2026 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex AI Model Garden - TimesFM 1.0 (CPU/GPU Deployment)

<table><tbody><tr>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fvertex-ai-samples%2Fmain%2Fnotebooks%2Fcommunity%2Fmodel_garden%2Fmodel_garden_timesfm_deployment_on_vertex.ipynb">
      <img alt="Google Cloud Colab Enterprise logo" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" width="32px"><br> Run in Colab Enterprise
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/model_garden/model_garden_timesfm_deployment_on_vertex.ipynb">
      <img alt="GitHub logo" src="https://github.githubassets.com/assets/GitHub-Mark-ea2971cee799.png" width="32px"><br> View on GitHub
    </a>
  </td>
</tr></tbody></table>

## Overview

This notebook demonstrates deploying TimesFM 1.0 to a Vertex AI Endpoint and
making online predictions for times series forecast.

### Objective

-   Deploy TimesFM 1.0 to a Vertex AI Endpoint.
-   Make predictions to the endpoint for times series forecast.

### Costs

This tutorial uses billable components of Google Cloud:

* Vertex AI
* Cloud Storage

Learn about [Vertex AI pricing](https://cloud.google.com/vertex-ai/pricing), [Cloud Storage pricing](https://cloud.google.com/storage/pricing), and use the [Pricing Calculator](https://cloud.google.com/products/calculator/) to generate a cost estimate based on your projected usage.

## Version History

This notebook demonstrates how to call the latest TimesFM endpoint. If you need to call an older version with different signatures, refer to the archived examples in the Appendix.

**2024-08-28 (latest)**:
- **Deploy Resource ID**: google/timesfm-v20240828
- Revised output signature for clarity.
- Added support for timestamps. TimesFM optionally accepts timestamps on the forecast context and incorporate them during making and formatting forecasts.
- Added supported for in-context residual linear model for covariates, both static and dynamic.

**2024-05-31**:
- **Deploy Resource ID**: google/timesfm
- Initial launch.

## Setup Google Cloud project

In [ ]:
# @markdown ### **Prerequisites**
# @markdown 1. [Make sure that billing is enabled for your project](https://cloud.google.com/billing/docs/how-to/modify-project).

# @markdown 2. **[Optional]** [Create a Cloud Storage bucket](https://cloud.google.com/storage/docs/creating-buckets) for storing experiment outputs. Set the BUCKET_URI for the experiment environment. The specified Cloud Storage bucket (`BUCKET_URI`) should be located in the same region as where the notebook was launched. Note that a multi-region bucket (eg. "us") is not considered a match for a single region covered by the multi-region range (eg. "us-central1"). If not set, a unique GCS bucket will be created instead.

BUCKET_URI = "gs://"  # @param {type:"string"}

# @markdown 3. **[Optional]** Set region. If not set, the region will be set automatically according to Colab Enterprise environment.

REGION = ""  # @param {type:"string"}

! git clone https://github.com/GoogleCloudPlatform/vertex-ai-samples.git

import datetime
import importlib
import os
import uuid
from typing import Tuple

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from google.cloud import aiplatform
from google.colab import output

common_util = importlib.import_module(
    "vertex-ai-samples.notebooks.community.model_garden.docker_source_codes.notebook_util.common_util"
)

models, endpoints = {}, {}


# Get the default cloud project id.
PROJECT_ID = os.environ["GOOGLE_CLOUD_PROJECT"]

# Get the default region for launching jobs.
if not REGION:
    if not os.environ.get("GOOGLE_CLOUD_REGION"):
        raise ValueError(
            "REGION must be set. See"
            " https://cloud.google.com/vertex-ai/docs/general/locations for"
            " available cloud locations."
        )
    REGION = os.environ["GOOGLE_CLOUD_REGION"]

# Enable the Vertex AI API and Compute Engine API, if not already.
print("Enabling Vertex AI API and Compute Engine API.")
! gcloud services enable aiplatform.googleapis.com compute.googleapis.com

# Cloud Storage bucket for storing the experiment artifacts.
# A unique GCS bucket will be created for the purpose of this notebook. If you
# prefer using your own GCS bucket, change the value yourself below.
now = datetime.datetime.now().strftime("%Y%m%d%H%M%S")
BUCKET_NAME = "/".join(BUCKET_URI.split("/")[:3])

if BUCKET_URI is None or BUCKET_URI.strip() == "" or BUCKET_URI == "gs://":
    BUCKET_URI = f"gs://{PROJECT_ID}-tmp-{now}-{str(uuid.uuid4())[:4]}"
    BUCKET_NAME = "/".join(BUCKET_URI.split("/")[:3])
    ! gsutil mb -l {REGION} {BUCKET_URI}
else:
    assert BUCKET_URI.startswith("gs://"), "BUCKET_URI must start with `gs://`."
    shell_output = ! gsutil ls -Lb {BUCKET_NAME} | grep "Location constraint:" | sed "s/Location constraint://"
    bucket_region = shell_output[0].strip().lower()
    if bucket_region != REGION:
        raise ValueError(
            "Bucket region %s is different from notebook region %s"
            % (bucket_region, REGION)
        )
print(f"Using this GCS Bucket: {BUCKET_URI}")

STAGING_BUCKET = os.path.join(BUCKET_URI, "temporal")
MODEL_BUCKET = os.path.join(BUCKET_URI, "timesfm")


# Initialize Vertex AI API.
print("Initializing Vertex AI API.")
aiplatform.init(project=PROJECT_ID, location=REGION, staging_bucket=STAGING_BUCKET)

# Gets the default SERVICE_ACCOUNT.
shell_output = ! gcloud projects describe $PROJECT_ID
project_number = shell_output[-1].split(":")[1].strip().replace("'", "")
SERVICE_ACCOUNT = f"{project_number}-compute@developer.gserviceaccount.com"
print("Using this default Service Account:", SERVICE_ACCOUNT)


# Provision permissions to the SERVICE_ACCOUNT with the GCS bucket
! gsutil iam ch serviceAccount:{SERVICE_ACCOUNT}:roles/storage.admin $BUCKET_NAME

! gcloud config set project $PROJECT_ID
! gcloud projects add-iam-policy-binding --no-user-output-enabled {PROJECT_ID} --member=serviceAccount:{SERVICE_ACCOUNT} --role="roles/storage.admin"
! gcloud projects add-iam-policy-binding --no-user-output-enabled {PROJECT_ID} --member=serviceAccount:{SERVICE_ACCOUNT} --role="roles/aiplatform.user"

# @markdown ### **Choose a prebuilt checkpoint**
# @markdown Here we specify where to get the model checkpoint. TimesFM
# @markdown pretrained checkpoints are by default saved under
# @markdown `gs://vertex-model-garden-public-{region}/timesfm` and indexed by
# @markdown the checkpoint version.

VERTEX_AI_MODEL_GARDEN_TIMESFM = "gs://vertex-model-garden-public-us/timesfm"  # @param {type:"string", isTemplate:true} ["gs://vertex-model-garden-public-us/timesfm", "gs://vertex-model-garden-public-eu/timesfm", "gs://vertex-model-garden-public-asia/timesfm"]
MODEL_VARIANT = "timesfm-1.0-200m"  # @param ["timesfm-1.0-200m"]
hf_model_id = "google/" + MODEL_VARIANT

print(
    "Copying TimesFM model artifacts from",
    f"{VERTEX_AI_MODEL_GARDEN_TIMESFM}/{MODEL_VARIANT}",
    "to",
    MODEL_BUCKET,
)

! gsutil -m cp -r -R $VERTEX_AI_MODEL_GARDEN_TIMESFM/$MODEL_VARIANT $MODEL_BUCKET

model_path_prefix = MODEL_BUCKET

## Deploy TimesFM to a Vertex AI Endpoint

In [ ]:
# @markdown This section uploads the prebuilt TimesFM model to Model Registry
# @markdown and deploys it to a Vertex AI Endpoint.
# @markdown It takes **approximately 20 minutes** to deploy. To view the source code of the serving docker, see [here](https://github.com/GoogleCloudPlatform/vertex-ai-samples/tree/fd5574fa1202727d7d587de25aff69450246965a/notebooks/community/model_garden/docker_source_codes/model_oss/timesfm).

# @markdown ### **Step 1: Set the checkpoint path**
# @markdown Leave this blank to load the checkpoint we copied over earlier.
# @markdown If you've brought your own checkpoint, specify its path here.
# @markdown
# @markdown **Note**: Most of the time you should leave it blank (as is)
# @markdown when you've chosen to use a prebuilt checkpoint.
# @markdown

custom_timesfm_model_uri = "gs://"  # @param {type: "string"}

if custom_timesfm_model_uri == "gs://" or not custom_timesfm_model_uri:
    print("Deploying prebuilt TimesFM model. ")
    checkpoint_path = model_path_prefix
else:
    print("Deploying custom TimesFM model.")
    checkpoint_path = custom_timesfm_model_uri
print(f"Loading checkpoint from {checkpoint_path}.")

# @markdown ### **Step 2: Choose the accelerator**
# @markdown Select the accelerator type to use to deploy the model.
# @markdown
# @markdown **Note**: Most of the time you can go with CPU only. TimesFM is
# @markdown fast even with the CPU backend. You can only consider GPU if you
# @markdown need a dedicated endpoint to handle large queries per second.
# @markdown
# @markdown **Note**: After deployment, take a look at the log to get
# @markdown the model / enpoint that you can use in another session.
# @markdown

accelerator_type = "CPU"  # @param ["CPU", "NVIDIA_L4"]
if accelerator_type == "NVIDIA_L4":
    machine_type = "g2-standard-4"
    accelerator_count = 1
elif accelerator_type == "CPU":
    accelerator_type = "ACCELERATOR_TYPE_UNSPECIFIED"
    machine_type = "n1-standard-8"
    accelerator_count = 0
else:
    raise ValueError(
        f"Recommended machine settings not found for: {accelerator_type}. To use"
        " another another accelerator, edit this code block to pass in an"
        " appropriate `machine_type`, `accelerator_type`, and"
        " `accelerator_count` to the deploy_model function by clicking `Show"
        " Code` and then modifying the code."
    )

if accelerator_type != "ACCELERATOR_TYPE_UNSPECIFIED":
    common_util.check_quota(
        project_id=PROJECT_ID,
        region=REGION,
        accelerator_type=accelerator_type,
        accelerator_count=accelerator_count,
        is_for_training=False,
    )

print("Quota is OK.")
# @markdown If you want to use other accelerator types not listed above,
# @markdown check other Vertex AI prediction supported accelerators and regions
# @markdown at https://cloud.google.com/vertex-ai/docs/predictions/configure-compute.
# @markdown You may need to manually set the `machine_type`, `accelerator_type`,
# @markdown and `accelerator_count` in the code by clicking `Show code` first.

# @markdown ### **Step 3: Set the maximum forecast horizon**
# @markdown We specify the maximum forecast horizon TimesFM will be queried on
# @markdown to compile its computation. The endpoint will always predict this
# @markdown number of time points in the future, possibly after being rounded
# @markdown up to the closest multiplier of the model output patch length.
# @markdown Make sure to set it to the potential maximum for your usecase.
horizon = 256  # @param {type:"number"}
print("Creating endpoint.")

SERVE_DOCKER_URI = "us-docker.pkg.dev/vertex-ai/vertex-vision-model-garden-dockers/jax-timesfm-serve:20240828_1036_RC00"
# @markdown Set `use_dedicated_endpoint` to False if you don't want to use [dedicated endpoint](https://cloud.google.com/vertex-ai/docs/general/deployment#create-dedicated-endpoint).
use_dedicated_endpoint = True  # @param {type:"boolean"}


def deploy_model(
    model_name: str,
    base_model_id: str,
    checkpoint_path: str,
    horizon: str,
    machine_type: str = "g2-standard-4",
    accelerator_type: str = "NVIDIA_L4",
    accelerator_count: int = 1,
    deploy_source: str = "notebook",
    use_dedicated_endpoint: bool = False,
) -> Tuple[aiplatform.Model, aiplatform.Endpoint]:
    """Creates a Vertex AI Endpoint and deploys TimesFM to the endpoint."""
    model_name_with_time = common_util.get_job_name_with_datetime(model_name)
    endpoint = aiplatform.Endpoint.create(
        display_name=f"{model_name_with_time}-endpoint",
        credentials=aiplatform.initializer.global_config.credentials,
        dedicated_endpoint_enabled=use_dedicated_endpoint,
    )

    if accelerator_type == "ACCELERATOR_TYPE_UNSPECIFIED":
        timesfm_backend = "cpu"
        accelerator_type = None
    elif accelerator_type.startswith("NVIDIA"):
        timesfm_backend = "gpu"
    else:
        timesfm_backend = "tpu"

    model = aiplatform.Model.upload(
        display_name=model_name_with_time,
        artifact_uri=checkpoint_path,
        serving_container_image_uri=SERVE_DOCKER_URI,
        serving_container_ports=[8080],
        serving_container_predict_route="/predict",
        serving_container_health_route="/health",
        serving_container_environment_variables={
            "MODEL_ID": base_model_id,
            "DEPLOY_SOURCE": deploy_source,
            "TIMESFM_HORIZON": str(horizon),
            "TIMESFM_BACKEND": timesfm_backend,
        },
        credentials=aiplatform.initializer.global_config.credentials,
        model_garden_source_model_name="publishers/google/models/timesfm",
    )
    print(
        f"Deploying {model_name_with_time} on {machine_type} with"
        f" {accelerator_count} {accelerator_type} GPU(s)."
    )
    model.deploy(
        endpoint=endpoint,
        machine_type=machine_type,
        accelerator_type=accelerator_type,
        accelerator_count=accelerator_count,
        deploy_request_timeout=1800,
        service_account=SERVICE_ACCOUNT,
        enable_access_logging=True,
        min_replica_count=1,
        sync=True,
        system_labels={
            "NOTEBOOK_NAME": "model_garden_timesfm_deployment_on_vertex.ipynb"
        },
    )
    return model, endpoint


models["timesfm"], endpoints["timesfm"] = deploy_model(
    model_name=f"timesfm-{MODEL_VARIANT}",
    base_model_id=hf_model_id,
    checkpoint_path=checkpoint_path,
    horizon=horizon,
    machine_type=machine_type,
    accelerator_type=accelerator_type,
    accelerator_count=accelerator_count,
    use_dedicated_endpoint=use_dedicated_endpoint,
)

## Query TimesFM

An endpoint prediction request looks like, with the `input` field mandatory and everything else optional.
```python
endpoint.predict(
    instances=[
        {
            "input": [0.0, 0.1, 0.2, ...],  # Mandatory
            "freq": 0,
            "horizon": 12,
            "timestamp": ["2024-01-01", "2024-01-02", ...],
            "timestamp_format": "%Y-%m-%d",
            "dynamic_numerical_covariates": {
                "dncov1": [1.0, 2.0, 1.5, ...],
                "dncov2": [3.0, 1.1, 2.4, ...],
            },
            "dynamic_categorical_covariates": {
                "dccov1": ["a", "b", "a", ...],
                "dccov2": [0, 1, 0, ...],
            },
            "static_numerical_covariates": {
                "sncov1": 1.0,
                "sncov2": 2.0,
            },
            "static_categorical_covariates": {
                "sccov1": "yes",
                "sccov2": "no",
            },
            "xreg_kwargs": {...},
        },
        {
            "input": [113.2, 15.0, 65.4],
            ...,
        },
        {
            "input": [0.0, 10.0, 20.0],
            ...,
        },
    ]
)
```


In [ ]:
# @markdown Create a helper function to visulize forecasts.


class Visualizer:
    def __init__(self, nrows, ncols):
        self.ncols = ncols
        self.num_images = nrows * ncols
        self.fig, self.axes = plt.subplots(
            nrows, ncols, figsize=(ncols * 4, nrows * 2.5)
        )
        self.axes = self.axes.flatten()
        self.index = 0

    def visualize_forecast(
        self,
        context: list[float],
        horizon_mean: list[float],
        ground_truth: list[float] | None = None,
        horizon_lower: list[float] | None = None,
        horizon_upper: list[float] | None = None,
        ylabel: str | None = None,
        title: str | None = None,
    ):
        plt_range = list(range(len(context) + len(horizon_mean)))
        self.axes[self.index].plot(
            plt_range,
            context + [np.nan for _ in horizon_mean],
            color="tab:cyan",
            label="context",
        )
        self.axes[self.index].plot(
            plt_range,
            [np.nan for _ in context] + horizon_mean,
            color="tab:red",
            label="forecast",
        )
        if ground_truth:
            self.axes[self.index].plot(
                list(range(len(context) + len(ground_truth))),
                [np.nan for _ in context] + ground_truth,
                color="tab:purple",
                label="ground truth",
            )
        if horizon_upper and horizon_lower:
            self.axes[self.index].plot(
                plt_range,
                [np.nan for _ in context] + horizon_upper,
                color="tab:orange",
                linestyle="--",
                label="forecast, upper",
            )
            self.axes[self.index].plot(
                plt_range,
                [np.nan for _ in context] + horizon_lower,
                color="tab:orange",
                linestyle=":",
                label="forecast, lower",
            )
            self.axes[self.index].fill_between(
                plt_range,
                [np.nan for _ in context] + horizon_upper,
                [np.nan for _ in context] + horizon_lower,
                color="tab:orange",
                alpha=0.2,
            )
        if ylabel:
            self.axes[self.index].set_ylabel(ylabel)
        if title:
            self.axes[self.index].set_title(title)
        self.axes[self.index].set_xlabel("time")
        self.axes[self.index].legend()
        self.index += 1

    def show(self):
        self.fig.tight_layout()
        self.fig.show()

In [ ]:
# @markdown We first check TimesFM on some sinusoidals. Notice that
# @markdown we are calling the endpoints minimally, with only the `input` field.
# Prepare the context. Notice each of them has a different context length.
# Note: this is strictly how the query should be structed:

instances = [
    {"input": np.sin(np.linspace(0, 20, 100)).tolist()},
    {"input": np.sin(np.linspace(0, 40, 500)).tolist()},
    {
        "input": (
            np.sin(np.linspace(0, 50, 300)) + np.sin(np.linspace(1, 71, 300)) * 0.5
        ).tolist()
    },
]

# Query the endpoint.
results = endpoints["timesfm"].predict(
    instances=instances,
    use_dedicated_endpoint=use_dedicated_endpoint,
)

viz = Visualizer(nrows=1, ncols=3)
viz.visualize_forecast(
    instances[0]["input"], results[0][0]["point_forecast"], title="Sinusoidal 1"
)
viz.visualize_forecast(
    instances[1]["input"], results[0][1]["point_forecast"], title="Sinusoidal 2"
)
viz.visualize_forecast(
    instances[2]["input"], results[0][2]["point_forecast"], title="Sinusoidal 3"
)
viz.show()

### Point forecast

We'll use real world dataset from Kaggle on the [daily temperatures in Delhi, India](https://www.kaggle.com/datasets/sumanthvrao/daily-climate-time-series-data/data). Set the Kaggle credentials following [these instructions](https://github.com/Kaggle/kaggle-api/blob/main/docs/README.md#api-credentials).


In [ ]:
# @markdown Prepare the dataset
! pip install kaggle

# Download and prepare the dataset
! kaggle datasets download sumanthvrao/daily-climate-time-series-data -p /tmp
! unzip /tmp/daily-climate-time-series-data.zip -d /tmp

output.clear()

data = pd.read_csv("/tmp/DailyDelhiClimateTrain.csv")
data

In [ ]:
# @markdown Query TimesFM and visualize the response

temperature = data.meantemp.to_list()
dates = data.date.to_list()
inputs = [temperature[0:200], temperature[300:600], temperature[700:1200]]
timestamps = [dates[0:200], dates[300:600], dates[700:1200]]
ground_truths = [
    temperature[200:300],
    temperature[600:700],
    temperature[1200:1300],
]

response = endpoints["timesfm"].predict(
    instances=[
        {
            "input": each_input,
            "horizon": 100,
            "timestamp": each_timestamp,
            "timestamp_format": "%Y-%m-%d",
        }
        for each_input, each_timestamp in zip(inputs, timestamps)
    ],
    use_dedicated_endpoint=use_dedicated_endpoint,
)

viz = Visualizer(nrows=1, ncols=3)

for task_i in range(3):
    viz.visualize_forecast(
        inputs[task_i],
        response[0][task_i]["point_forecast"][:100],
        ground_truth=ground_truths[0],
        title=f"Daily temperature in Delhi, India, Task {task_i+1}",
        ylabel="Temperature (°C)",
    )
viz.show()

Notice we are calling by
```python
response = endpoint.predict(
    instances=[
        {
            "input": each_input,
            "horizon": 50,
            "timestamp": each_timestamp,
            "timestamp_format": "%Y-%m-%d",
        }
        for each_input, each_timestamp in zip(inputs, timestamps)
    ]
)
```

Here the input fields mean respectively that
- `input`: the context of the time series
- `horizon`: an optional integer indicating how long the forecast horizon for this task is. If not given, we'll use the maximum horizon length specified when creating the endpoint.
- `timestamp`: an optional list of [ISO_8601](https://en.wikipedia.org/wiki/ISO_8601) time strings corresponding to each value in `input`.
 - If provided, TimesFM will infer the `freq` field accordingly.
 - If there are skipped timestamps, TimesFM will linearly interpolate the missing ones before making the forecast.
 - TimesFM will also infer the future timestamps, and apply [strftime](https://docs.python.org/3/library/datetime.html#strftime-and-strptime-format-codes) with `timestamp_format` to format the return.
- `freq`: since `timestamp` is given we inferred the frequency as `daily`. Otherwise it would be helpful to provide this `freq` integer direct. Rule of thumb: `0` for everything up till daily, `1` for weekly and monthly, and `2` for quarterly and yearly.

This `response` is structured that:
* `response[0][i]` is the forecast result of the ith input inside `instances`.
* `response[0][i]` has the following keys:
 - `timestamp`: the timestamps of the forecast horizon (since we provided `timestamp` during the call).
 - `point_forecast`: the mean point forecast.
 - `mean`: same as `point_forecast`.
 - `p{a}` for `a` in 10, 20, ..., 90: the `a`th percentile of TimesFM forecast.
   

### Anomaly detection

As of checkpoint TimesFM-1.0-200m, TimesFM is capable of outputing quantile forecasts as well. These are uncalibrated forecasts and are experimental. But feel free to experiment and to see what you can do with them.

Here we show how these outputs can potentially serve as anomaly detectors, when we define the anomaly as something beyond a certain range of TimesFM forecasts. In this example we are drawing bands defined by the 30th and the 70th percentiles on the same tasks we did in the last section. Anything outside of the bands could be an "anomaly".

In [ ]:
# @markdown Visualize the response

viz = Visualizer(nrows=1, ncols=3)
for task_i in range(3):
    viz.visualize_forecast(
        inputs[task_i],
        response[0][task_i]["point_forecast"],
        ground_truth=ground_truths[0],
        horizon_lower=response[0][task_i]["p30"],
        horizon_upper=response[0][task_i]["p70"],
        title=f"Daily temperature in Delhi, India, Task {task_i+1}",
        ylabel="Temperature (°C)",
    )
viz.show()

### Covariate support

TimesFM can incorporate static and dynamic covariates to better the forecast. The native function is demonstrated in this [notebook on GitHub](https://github.com/google-research/timesfm/blob/master/notebooks/covariates.ipynb), which also has a few examples introducing covariates, their definitions and how they can be useful.

For the Model Garden enpoint we've created awrapper to request the forecast with covaraties directly.


In [ ]:
# @markdown Create synthetic data, call TimesFM with covariates, and visualize
# @markdown the outputs.
# @markdown Notice the forecasts with covariates capture the nuanced spikes.

start = datetime.datetime.fromisoformat("2024-01-01 11:30:20")
dummy_timestamps = [start + datetime.timedelta(minutes=10) * k for k in range(1000)]
dummy_sin = np.sin(np.linspace(0.0, 300.0, 1000))
dummy_dynamic = np.random.uniform(size=1000)
dummy_dynamic_cat = np.random.binomial(n=1, p=0.3, size=1000)
dummy_mix = dummy_sin + dummy_dynamic * 0.5 + dummy_dynamic_cat * 0.5
dummy_mix_yes = dummy_mix + 0.5
dummy_mix_no = dummy_mix - 0.5

cov_instances = [
    {
        "input": dummy_mix_yes[:200].tolist(),
        "dynamic_numerical_covariates": {
            "temperature_forecast": dummy_dynamic[:240].tolist(),
        },
        "dynamic_categorical_covariates": {
            "will_it_rain": dummy_dynamic_cat[:240].tolist(),
        },
        "static_numerical_covariates": {"not_useful_cov1": 0.2},
        "static_categorical_covariates": {"take_vitamin_c": "yes"},
        "timestamp": [k.strftime("%Y-%m-%d %H:%M:%S") for k in dummy_timestamps[:200]],
    },
    {
        "input": dummy_mix_yes[300:400].tolist(),
        "dynamic_numerical_covariates": {
            "temperature_forecast": dummy_dynamic[300:440].tolist()
        },
        "dynamic_categorical_covariates": {
            "will_it_rain": dummy_dynamic_cat[300:440].tolist()
        },
        "static_numerical_covariates": {"not_useful_cov1": 0.1},
        "static_categorical_covariates": {"take_vitamin_c": "yes"},
        "timestamp": [
            k.strftime("%Y-%m-%d %H:%M:%S") for k in dummy_timestamps[300:400]
        ],
    },
    {
        "input": dummy_mix_no[500:800].tolist(),
        "dynamic_numerical_covariates": {
            "temperature_forecast": dummy_dynamic[500:820].tolist()
        },
        "dynamic_categorical_covariates": {
            "will_it_rain": dummy_dynamic_cat[500:820].tolist()
        },
        "static_numerical_covariates": {"not_useful_cov1": 0.3},
        "static_categorical_covariates": {"take_vitamin_c": "no"},
        "timestamp": [
            k.strftime("%Y-%m-%d %H:%M:%S") for k in dummy_timestamps[500:800]
        ],
    },
]

response = endpoints["timesfm"].predict(
    instances=cov_instances,
    use_dedicated_endpoint=use_dedicated_endpoint,
)

no_cov_instances = [{"input": task["input"], "horizon": 40} for task in cov_instances]
no_cov_response = endpoints["timesfm"].predict(
    instances=no_cov_instances,
    use_dedicated_endpoint=use_dedicated_endpoint,
)

viz = Visualizer(nrows=3, ncols=2)
for task_i, (per_input, per_gt) in enumerate(
    [
        (dummy_mix_yes[:200], dummy_mix_yes[200:240]),
        (dummy_mix_yes[300:400], dummy_mix_yes[400:440]),
        (dummy_mix_no[500:800], dummy_mix_no[800:820]),
    ]
):
    viz.visualize_forecast(
        per_input.tolist(),
        response[0][task_i]["point_forecast"],
        ground_truth=per_gt.tolist(),
        title=f"Task {task_i} WITH covariates",
    )
    viz.visualize_forecast(
        per_input.tolist(),
        no_cov_response[0][task_i]["point_forecast"],
        ground_truth=per_gt.tolist(),
        title=f"Task {task_i} WITHOUT covariates",
    )
viz.show()

In this example, for each task the signature is
```python
 {
    "input": dummy_mix_yes[:200].tolist(),
    "dynamic_numerical_covariates": {
        "temperature_forecast": dummy_dynamic[:240].tolist(),
    },
    "dynamic_categorical_covariates": {
        "will_it_rain": dummy_dynamic_cat[:240].tolist(),
    },
    "static_numerical_covariates": {"not_useful_cov1": 0.2},
    "static_categorical_covariates": {"take_vitamin_c": "yes"},
    "timestamp": [
        k.strftime("%Y-%m-%d %H:%M:%S") for k in dummy_timestamps[:200]
    ],
}
```

**Notice**:

1. We prefer multiple tasks in one `predict` call. As our covariate support needs batched inputs, it helps to have more tasks with the same set of covariates for more accurate forecast.

2. We make it mandatory that the dynamic covariates need to cover both the forecast context and horizon. For example, if you have a 7-day history and want to forecast 7 days in the future, all dynamic covariates should have 14 values covering both time periods.

3. You can add more covariates into the correspoinding dictionary if needed - just make sure that all tasks in the same request share the same set of covariates.

4. The `horizon` is not provided, since TimesFM can infer the correct `horizon` based on the length of the horizon part of the dynamic covariates. If only static covariates are available it is still worth considering adding back the `horizon`.

## Clean up resources

In [ ]:
# @title Delete the models and endpoints
# @markdown  Delete the experiment models and endpoints to recycle the resources
# @markdown  and avoid unnecessary continuous charges that may incur.

# Undeploy model and delete endpoint.
for endpoint in endpoints.values():
    endpoint.delete(force=True)

# Delete models.
for model in models.values():
    model.delete()

delete_bucket = False  # @param {type:"boolean"}
if delete_bucket:
    ! gsutil -m rm -r $BUCKET_NAME

## Appendix

Here you can find the examples for calling older versions of TimesFM, indexed by the corresponding **Resource ID** when being deployed.

In [ ]:
# @markdown google/timesfm

# import pandas as pd

# data = pd.read_csv("/tmp/DailyDelhiClimateTrain.csv")
# data

# # We manually prepare 3 forecast tasks:
# # 1. Use day 0 - 199 to forecast day 200-299.
# # 2. Use day 300 - 599 to forecast day 600-699.
# # 3. Use day 700 - 1200 to forecast day 1200-1299.
# temperature = data.meantemp.to_list()
# inputs = [temperature[0:200], temperature[300:600], temperature[700:1200]]
# ground_truths = [
#     temperature[200:300],
#     temperature[600:700],
#     temperature[1200:1300],
# ]
# response = endpoint.predict(
#     instances=[{"input": each_input, "freq": 0} for each_input in inputs]
# )
# response[0][0].keys()

# # Visualize the response
# viz = Visualizer(nrows=1, ncols=3)
# for task_i in range(3):
#     viz.visualize_forecast(
#         inputs[task_i],
#         response[0][task_i]["point_forecast"][:100],
#         ground_truth=ground_truths[0],
#         title=f"Daily temperature in Delhi, India, Task {task_i+1}",
#         ylabel="Temperature (°C)",
#     )
# viz.show()

# # Visualize the response
# viz = Visualizer(nrows=1, ncols=3)
# for task_i in range(3):
#     viz.visualize_forecast(
#         inputs[task_i],
#         response[0][task_i]["point_forecast"][:100],
#         ground_truth=ground_truths[0],
#         horizon_lower=[x[3] for x in response[0][task_i]["quantile_forecast"]][:100],
#         horizon_upper=[x[7] for x in response[0][task_i]["quantile_forecast"]][:100],
#         title=f"Daily temperature in Delhi, India, Task {task_i+1}",
#         ylabel="Temperature (°C)",
#     )
# viz.show()